# InitialSetup

In [1]:
import os
import shutil

if not os.path.exists('gen_data_set/'):
    raise Exception("Generated dataset not found, pls run 'SetupDataset' first.")
    
if os.path.exists('OUTPUT/'):
    shutil.rmtree('OUTPUT/')
    
os.makedirs('OUTPUT/')

# Loading dataset

In [2]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255, brightness_range=[0.2,1.0], shear_range=0.2,
                                    fill_mode = 'nearest', width_shift_range=0.2, rotation_range=40,
                                   height_shift_range=0.2, horizontal_flip=True, zoom_range=0.2)


test_gen = ImageDataGenerator(rescale=1./255)

training_data = train_gen.flow_from_directory('gen_data_set/train', target_size=(265,256), 
                                                 batch_size=32, class_mode='binary')

test_data = test_gen.flow_from_directory('gen_data_set/test', target_size=(256, 256), 
                                            batch_size=32, class_mode='binary')

2023-08-27 17:10:45.987032: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 1890 images belonging to 2 classes.
Found 1573 images belonging to 2 classes.


# Training model from scratch

In [3]:
from keras.models import Sequential
from keras.layers import MaxPooling2D, Convolution2D, Flatten, Dense

classifier = Sequential([
    Convolution2D(64,3,3, input_shape=(256,256,3), activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Convolution2D(32,3,3, activation='relu'),
    MaxPooling2D(pool_size = (2,2)),
    
    Flatten(),
    
    Dense(256, activation = 'relu'),
    Dense(1, activation = 'sigmoid')
])

In [4]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [5]:
steps_per_epoch = len(training_data)
validation_steps = len(test_data)

classifier.fit(training_data, epochs=90, steps_per_epoch=steps_per_epoch, workers=1,
                    validation_data=test_data, validation_steps=validation_steps)

Epoch 1/90


2023-08-27 17:10:49.393495: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


60/60 [==============================] - ETA: 0s - loss: 0.5216 - accuracy: 0.7556

2023-08-27 17:11:35.649446: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


60/60 [==============================] - 56s 908ms/step - loss: 0.5216 - accuracy: 0.7556 - val_loss: 0.1329 - val_accuracy: 0.9472
Epoch 2/90
60/60 [==============================] - 60s 1s/step - loss: 0.2637 - accuracy: 0.9111 - val_loss: 0.1633 - val_accuracy: 0.9574
Epoch 3/90
60/60 [==============================] - 54s 900ms/step - loss: 0.2371 - accuracy: 0.9190 - val_loss: 0.2373 - val_accuracy: 0.9479
Epoch 4/90
60/60 [==============================] - 52s 875ms/step - loss: 0.1845 - accuracy: 0.9402 - val_loss: 0.1309 - val_accuracy: 0.9669
Epoch 5/90
60/60 [==============================] - 52s 866ms/step - loss: 0.1588 - accuracy: 0.9455 - val_loss: 0.1423 - val_accuracy: 0.9669
Epoch 6/90
60/60 [==============================] - 52s 862ms/step - loss: 0.1703 - accuracy: 0.9354 - val_loss: 0.0933 - val_accuracy: 0.9758
Epoch 7/90
60/60 [==============================] - 52s 867ms/step - loss: 0.1759 - accuracy: 0.9365 - val_loss: 0.1857 - val_accuracy: 0.9599
Epoch 8/90
60

60/60 [==============================] - 52s 863ms/step - loss: 0.0675 - accuracy: 0.9757 - val_loss: 0.0638 - val_accuracy: 0.9784
Epoch 59/90
60/60 [==============================] - 52s 865ms/step - loss: 0.0557 - accuracy: 0.9820 - val_loss: 0.0611 - val_accuracy: 0.9816
Epoch 60/90
60/60 [==============================] - 51s 859ms/step - loss: 0.0623 - accuracy: 0.9810 - val_loss: 0.0735 - val_accuracy: 0.9771
Epoch 61/90
60/60 [==============================] - 52s 870ms/step - loss: 0.0638 - accuracy: 0.9799 - val_loss: 0.0517 - val_accuracy: 0.9854
Epoch 62/90
60/60 [==============================] - 52s 863ms/step - loss: 0.0767 - accuracy: 0.9730 - val_loss: 0.0645 - val_accuracy: 0.9809
Epoch 63/90
60/60 [==============================] - 52s 866ms/step - loss: 0.0774 - accuracy: 0.9720 - val_loss: 0.0793 - val_accuracy: 0.9765
Epoch 64/90
60/60 [==============================] - 52s 864ms/step - loss: 0.0634 - accuracy: 0.9810 - val_loss: 0.0639 - val_accuracy: 0.9784
Epoc

In [6]:
classifier.save("OUTPUT/ScratchModel.h5")

In [7]:
_, accuracy = classifier.evaluate(test_data, steps = validation_steps)

 1/50 [..............................] - ETA: 7s - loss: 0.0301 - accuracy: 0.9688

2023-08-27 18:28:58.259992: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


50/50 [==============================] - 8s 167ms/step - loss: 0.0624 - accuracy: 0.9816


In [8]:
print('ScratchModel accuracy :',accuracy)

ScratchModel accuracy : 0.9815638661384583
